In [11]:
"""instances_folder = Path.cwd().parent / 'instances'
filename = f'inst05.dat'
file_path = instances_folder / filename

data = read_from_file(file_path)
couriers = data[0][0]
items = data[1][0]
l = data[2]
max_item = max(l)
s = data[3]
D = np.array(data[4:])
nodes = items+1
"""

In [14]:
from pathlib import Path
import numpy as np
import json
import os
from datetime import datetime
from pulp import LpProblem, LpMinimize, LpVariable, LpBinary, lpSum, PULP_CBC_CMD, LpStatus,LpInteger


def read_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    data = []
    for line in lines:
        numbers = list(map(int, line.split()))
        data.append(numbers)
    
    return data

def extract_solution(X, nodes, couriers):
    sol = [[] for _ in range(couriers)]
    
    for k in range(couriers):
        current_node = nodes-1 # Start from the depot (now the last node)
        route = []  # Initialize route
        
        while True:
            next_node = None
            for j in range(nodes):
                if X[current_node][j][k].varValue == 1:
                    next_node = j
                    break
            
            # If there is no next node or the next node is the depot and not the start of the route, break the loop
            if next_node == nodes - 1 :
                break
            
            route.append(next_node+1)
            current_node = next_node

        sol[k] = route

    return sol

def compute_total_distance(D, route, nodes):
    """
    Compute the total distance for a given route where the route starts and ends at the depot (node nodes-1).
    
    Parameters:
    - D (list of list of int): The distance matrix.
    - route (list of int): The sequence of nodes in the route (excluding the depot).
    
    Returns:
    - int: The total distance of the route including the depot.
    """
    depot = nodes-1  # Depot is at the last node (nodes-1)
    total_distance = 0
    
    # Add distance from depot to the first node in the route
    #if route:
    #    total_distance += D[depot][route[0]]
    
    # Add distance between consecutive nodes in the route
    #for i in range(len(route) - 1):
    #    total_distance += D[route[i]][route[i + 1]]
        # Add distance between consecutive nodes in the route
    if len(route) > 1:
        total_distance += np.sum(D[route[:-1], route[1:]])
    # Add distance from the last node in the route back to the depot
    #if route:
    #        total_distance += D[route[-1]][depot]
    
    return total_distance


def sort_couriers_by_capacity(capacities):
    """
    Sort couriers by their capacities in descending order.

    Parameters:
    - capacities (list of int): List of capacities for each courier.

    Returns:
    - sorted_indices (numpy array): Indices of couriers sorted by capacity.
    - sorted_capacities (numpy array): Capacities sorted in descending order.
    """
    capacities = np.array(capacities)
    sorted_indices = np.argsort(-capacities)  # Sort in descending order
    sorted_capacities = capacities[sorted_indices]

    return sorted_indices, sorted_capacities

def set_Z_low_bound(D):
    """
    Lower bound for the objective function rho.
    It is the longest dep->item->dep route possible among the items.
    """
    last_row = D[-1]
    last_column = D[:, -1]
    value1 = last_column[np.argmax(last_row)] + max(last_row)
    value2 = last_row[np.argmax(last_column)] + max(last_column)
    lb = max(value1, value2)
    return lb

# compute up bound max distance
def set_Z_up_bound(D, nodes):
    """
    Upper bound for the distance that each courier can travel.
    It is the distance of the path [1,2,...,n] (a courier brings all items at once).
    """
    up_bound = D[nodes-1, 0] + np.sum(np.diag(D[0:nodes-1, 1:nodes])) + D[nodes-1, nodes-1]
    return up_bound


def set_d_low_bound(D):
    """
    Lower bound for the distance that each courier can travel.
    It is the shortest dep->item->dep route possible among the items.
    """
    last_row = D[-1]
    last_column = D[:, -1]
    last_column = last_column[last_column != 0]
    last_row = last_row[last_row != 0]
    value1 = last_column[np.argmin(last_row)] + min(last_row)
    value2 = last_row[np.argmin(last_column)] + min(last_column)
    d_low_bound = min(value1, value2)
    return d_low_bound

def test_instance(file_path, filename, timelimit=300, result_folder='res',print_path=False):
    data = read_from_file(file_path)
    couriers = data[0][0]
    items = data[1][0]
    l = data[2]
    s = data[3]
    D = np.array(data[4:])
    nodes = items+1
        
    prob = LpProblem("Vehicle_Routing_Problem", LpMinimize)

    start_time = datetime.now()

    lb = set_Z_low_bound(D)
    ub = set_Z_up_bound(D, nodes)
    X = LpVariable.dicts("X", (range(nodes), range(nodes), range(couriers)), lowBound=0, upBound=1,cat=LpBinary)

    # Define the maximum distance variable Z
    Z = LpVariable("Z", lowBound=lb, upBound=ub, cat=LpInteger)
    #Z = LpVariable("Z", upBound=ub, cat=LpInteger)

    # MTZ Constraints
    u = LpVariable.dicts("u", range(items), lowBound=0, upBound=items, cat=LpInteger)

    # No self loop
    for k in range(couriers): 
        for i in range(nodes):
            prob += X[i][i][k] == 0
        
    #1 Vehicle leaves node that it enters
    for k in range(couriers):
        for j in range(nodes): 
            prob += lpSum(X[i][j][k] for i in range(nodes)) == lpSum(X[j][i][k] for i in range(nodes))
        
    # Ensure that every node except the depot is entered exactly once
    for j in range(items):  # Skipping the depot at index nodes-1
        prob += lpSum(X[i][j][k] for i in range(nodes) for k in range(couriers)) == 1

    # Ensure every vehicle leaves the depot
    for k in range(couriers):
        prob += lpSum(X[nodes-1][j][k] for j in range(items)) == 1

    # Capacity constraint
    for k in range(couriers):
        prob += lpSum(X[i][j][k] * s[j] for i in range(nodes) for j in range(nodes - 1)) <= l[k]

    # Add constraints for Z
    for k in range(couriers):
        prob += lpSum(D[i][j] * X[i][j][k] for i in range(nodes) for j in range(nodes)) <= Z

    # Eliminating subtours
    for k in range(couriers):
        for i in range(nodes - 1):  # Excluding depot at index nodes-1
            for j in range(nodes - 1):  # Excluding depot at index nodes-1
                prob += u[i] - u[j] + (nodes - 1) * X[i][j][k] <= nodes - 2

    prob += Z
    
 
    preprocessing_time = (datetime.now() - start_time).total_seconds()
    remaining_time = max(0, timelimit - preprocessing_time)
    # Use the CBC solver with a time limit
    solver = PULP_CBC_CMD(timeLimit=remaining_time)

    # Solve the problem
    prob.solve(solver)

    end_time = datetime.now()

    actual_runtime = (end_time - start_time).total_seconds()
    print(actual_runtime)
    time = min(int(actual_runtime), 300)


    status = LpStatus[prob.status]
    if status != 'Optimal':
        print("Instance not solved")
    else:
        print(status)
        # Extract solution
        sol = extract_solution(X, nodes, couriers)

        optimal = LpStatus[prob.status] == 'Optimal'
        if time>=300:
            optimal = False

        result = {
            "solver_name": {
                "time": time,
                "optimal": optimal,
                "obj": Z.varValue,
                "sol": sol
            }
        }

        # Ensure result directory exists
        os.makedirs(result_folder, exist_ok=True)
        
        # Save result to JSON file
        json_path = os.path.join(result_folder, f"{filename.split('.')[0]}.json")
        with open(json_path, 'w') as json_file:
            json.dump(result, json_file, indent=4)

    return time, Z.varValue

In [15]:
# Define the folder containing the instances
instances_folder = Path.cwd().parent / 'instances'

# List of specific instance numbers you want to test
instance_numbers = list(range(1, 11)) + [13, 16]

# Loop through each instance number
for i in instance_numbers:
    
    # Format the filename to match the pattern 'instXX.dat'
    filename = f'inst{i:02d}.dat'
    file_path = instances_folder / filename
    
    # Test the instance using your test_instance function
    time, obj = test_instance(file_path, filename, timelimit=300, result_folder='res/MIP2')
    
    # Print the results for each instance
    print(f"Instance: {filename} - Time: {time}s - Objective Value: {obj}")


0.623
Optimal
Instance: inst01.dat - Time: 0s - Objective Value: 14.0
0.409999
Optimal
Instance: inst02.dat - Time: 0s - Objective Value: 226.0
0.271002
Optimal
Instance: inst03.dat - Time: 0s - Objective Value: 12.0
0.20059
Optimal
Instance: inst04.dat - Time: 0s - Objective Value: 220.0
0.019001
Optimal
Instance: inst05.dat - Time: 0s - Objective Value: 206.0
0.103002
Optimal
Instance: inst06.dat - Time: 0s - Objective Value: 322.0
124.06959
Optimal
Instance: inst07.dat - Time: 124s - Objective Value: 167.0
0.200997
Optimal
Instance: inst08.dat - Time: 0s - Objective Value: 186.0
0.327997
Optimal
Instance: inst09.dat - Time: 0s - Objective Value: 436.0
0.384005
Optimal
Instance: inst10.dat - Time: 0s - Objective Value: 244.0
300.202789
Optimal
Instance: inst13.dat - Time: 300s - Objective Value: 704.0
298.323814
Optimal
Instance: inst16.dat - Time: 298s - Objective Value: 315.0
